In [12]:
from neo4j import GraphDatabase
import DATABASE_CONFIG

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j://localhost"
AUTH = ("neo4j", DATABASE_CONFIG.DB_PASSWORD)

In [13]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    driver.execute_query(
        "CREATE CONSTRAINT id_unidade_conservacao_constraint FOR (r:UnidadeConservacao) REQUIRE r.id_unidade_conservacao IS UNIQUE"
    )
    driver.execute_query(
        "CREATE CONSTRAINT nome_unidade_conservacao_constraint FOR (r:UnidadeConservacao) REQUIRE r.nome IS UNIQUE"
    )

In [14]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    driver.execute_query(
        "LOAD CSV WITH HEADERS FROM 'file:///unidade_conservacao.csv' AS csvLine "
        "CREATE (r:UnidadeConservacao {id_unidade_conservacao: toInteger(csvLine.id_unidade_conservacao), nome: csvLine.nome, categoria: csvLine.categoria, "
        "esfera_administrativa: csvLine.esfera_administrativa, tipo: csvLine.tipo, area_amazonia_ha: toFloat(csvLine.area_amazonia_ha)})"
    )

In [15]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    driver.execute_query(
        "LOAD CSV WITH HEADERS FROM 'file:///unidade_conservacao.csv' AS csvLine "
        "MATCH (e:Estado {id_estado: toInteger(csvLine.id_estado)}) "
        "MATCH (uc:UnidadeConservacao {id_unidade_conservacao: toInteger(csvLine.id_unidade_conservacao)}) "
        "MERGE (uc)-[:Pertence]->(e) "
        )

In [16]:
try:
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()
        driver.execute_query(
            "LOAD CSV WITH HEADERS FROM 'file:///cidade_unidade_conservacao.csv' AS csvLine "
            "MATCH (c:Cidade {id_cidade: toInteger(csvLine.cidade_id_cidade)}) "
            "MATCH (u:UnidadeConservacao {id_unidade_conservacao: toInteger(csvLine.unidade_conservacao_id_unidade_conservacao)}) "
            "MERGE (c)-[:Abrange]->(u) "
            "MERGE (u)-[:Abrange]->(c)"
        )
    print("Relacionamentos criados com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Relacionamentos criados com sucesso!
